## **Notebook Summary**

### **Purpose**
This notebook evaluates and visualizes the performance of a clinical trial-patient matching system ("trial_matches.json"), comparing system-assigned trial suitability verdicts against curated gold-standard labels.

---

### **Key Steps and Logic**

#### **1. Data Loading**
- Loads:
  - **Gold/reference** patient-trial pairs and scores from `test.tsv`.
  - **Patient queries** and **trial corpus** from `queries.jsonl` and `corpus.jsonl`.
  - **Trial matches** (detailed matching outputs) from `matched/trial_matches.json` (contains verdicts, reasoning about inclusion/exclusion).

#### **2. Flatten & Normalize Trial Matches**
- Iterates through `trial_matches.json` to **flatten nested structures** into a DataFrame:
  - Columns include: `query-id`, `corpus-id`, model trial verdict (`match_verdict`), reasoning, inclusion/exclusion flags, and detailed eligibility explanations.

#### **3. Merge Gold with System Predictions**
- Merges gold-standard pairs (`df_tsv`) with matched verdicts (`df_trial_matches`) by query and trial IDs.
- Ensures **all gold pairs are present**, filling unmatched ones with NaN for later filtering/checks.

#### **4. Label Harmonization**
- Maps system verdicts (“STRONG MATCH”, “POSSIBLE MATCH”, “WEAK MATCH”, “UNSUITABLE”) into numeric label space {0, 1, 2} for comparison:
  - 2: Highly likely ("STRONG MATCH" or "POSSIBLE MATCH")
  - 1: Would consider ("WEAK MATCH")
  - 0: Would not refer ("UNSUITABLE")
- Aligns ground-truth labels from gold file into the same space.

#### **5. Evaluation**
- **Omissions:** Reports count/samples of pairs where there is no system verdict (excluded from scoring).
- **Metrics:**
  - Computes **confusion matrix** (tabular and heatmap) to visualize agreement.
  - Calculates **macro-averaged F1 score** and displays a full classification report (precision/recall per label).
- **Visualization:** Plots the confusion matrix with Seaborn for a clear model performance summary.

#### **6. Inspection & Quality Control**
- Provides options to display merged/evaluated DataFrame (with explanations/reasoning), as well as any omitted/missing rows.

#### **7. Display/Output**
- Configures pandas for wide displays, allowing direct inspection of large slices of merged data and reasoning fields.

---

### **Outputs**
- **Metrics for Model QA:** Confusion matrix, macro F1, precision/recall for each decision level.
- **Visualization:** Color-coded confusion matrix heatmap for rapid error pattern assessment.
- **Inspection:** Full merged DataFrame can be displayed, supporting qualitative analysis via system reasoning and verdicts.

---

### **Intended Use**
- **Model validation and debugging:** Evaluate system-patient-to-trial assignment quality.
- **Insight into errors:** Directly reveals misclassifications and their reasoning by exploring DataFrame rows.
- **Transparent reporting:** Well-suited for sharing system strengths and weaknesses with stakeholders or during clinical IR system development.

---

### **Technical/Architectural Notes**
- **Robust merging:** Ensures all reference pairs are considered, not just those found by the matcher.
- **Extendable:** Can be easily fit for other datasets or more granular scoring/justification analysis.
- **User-friendly:** Designed for rapid exploratory analysis, including straightforward displays for both summary and raw data.

In [ ]:
import os
import json
import pandas as pd
import numpy as np

In [ ]:
# Set pandas display options 
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 400)

In [ ]:
import os
import pandas as pd
import json

# Set up paths
DATA_DIR = "../data/sigir2016/processed_cut"
MATCHED_DIR = "../data/sigir2016/matched"

TSV_FILE = os.path.join(DATA_DIR, "test.tsv")
QUERIES_FILE = os.path.join(DATA_DIR, "queries.jsonl")
CORPUS_FILE = os.path.join(DATA_DIR, "corpus.jsonl")
TRIAL_MATCHES_FILE = os.path.join(MATCHED_DIR, "trial_matches.json")

# Load main input data
df_tsv = pd.read_csv(TSV_FILE, sep='\t')
df_queries = pd.read_json(QUERIES_FILE, lines=True)
df_corpus = pd.read_json(CORPUS_FILE, lines=True)

In [ ]:
print(f" df_tsv {len(df_tsv)} df_queries {len(df_queries)} df_corpus {len(df_corpus)}")

In [ ]:
# Glimpse at dataframes:
display(df_tsv.head())
display(df_queries.head())
display(df_corpus.head())
# display(trial_matches_long.head())

# ---- Now you can proceed with analysis & evaluation below!

In [ ]:
TRIAL_MATCHES_FILE = os.path.join(MATCHED_DIR, "trial_matches.json")

# Load the JSON as a list of dicts
with open(TRIAL_MATCHES_FILE, "r") as f:
    trial_matches = json.load(f)

# Flattening loop
records = []
for patient_dict in trial_matches:
    patient_id = patient_dict['patient_id']
    for eval_ in patient_dict['trial_evaluations']:
        trial_id = eval_['trial_id']
        trial_title = eval_.get('trial_title')
        scoring = eval_.get('scoring_result', {})
        inclusion = eval_.get('inclusion_result', {})
        exclusion = eval_.get('exclusion_result', {})
        records.append({
            'query-id': patient_id,
            'corpus-id': trial_id,
            'trial_title': trial_title,
            # From scoring_result
            'score': scoring.get('score'),
            'match_verdict': scoring.get('verdict'),
            'match_reasoning': scoring.get('reasoning'),
            # From exclusion_result
            'exclusion_verdict': exclusion.get('verdict'),
            'exclusion_reason': exclusion.get('reason'),
            # From inclusion_result
            'inclusion_verdict': inclusion.get('verdict'),
            'inclusion_missing_info': inclusion.get('missing_information'),
            'inclusion_unmet_criteria': inclusion.get('unmet_criteria'),
            'inclusion_reasoning': inclusion.get('reasoning'),
        })

# Create DataFrame
df_trial_matches = pd.DataFrame(records)

# Preview
# pd.set_option('display.max_colwidth', 200)
# display(df_trial_matches.head())

In [ ]:
df_trial_matches['score'] = df_trial_matches['score'].fillna(0)
df_trial_matches['match_verdict'] = df_trial_matches['match_verdict'].fillna("WOULD NOT REFER")

In [ ]:
len(df_trial_matches)

In [ ]:
# df_tsv: gold/reference pairs
# df_trial_matches: detailed match+reasoning info from trial_matches.json

# Merge, keeping all gold standard pairs and including trial_matches info when available:
merged_df = df_tsv.merge(df_trial_matches, on=['query-id', 'corpus-id'], how='left')

In [ ]:
# 1. Merge queries info (like 'text') into merged_df
merged_df = merged_df.merge(
    df_queries.rename(columns={'_id': 'query-id'}),
    on='query-id',
    how='left'
)

# 2. Merge corpus info (like 'title', 'metadata') into merged_df
merged_df = merged_df.merge(
    df_corpus.rename(columns={'_id': 'corpus-id'}),
    on='corpus-id',
    how='left'
)
# Drop the 'title' column
merged_df = merged_df.drop(columns=['title'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import pandas as pd

verdict_map = {
    "HIGHLY LIKELY TO REFER": 2,
    "WOULD CONSIDER REFERRAL": 1,
    "WOULD NOT REFER": 0,
    "None": 0,
}

merged_df['predicted_label'] = merged_df['match_verdict'].map(verdict_map)#.fillna(0).astype(int)
merged_df['true_label'] = merged_df['score_x']

labels = [0, 1, 2]
label_names = [
    "0) WOULD NOT REFER",
    "1) WOULD CONSIDER REFERRAL",
    "2) HIGHLY LIKELY TO REFER"
]

n_missing = merged_df['match_verdict'].isna().sum()
print(f"Rows with missing match_verdict (excluded from evaluation): {n_missing}")

eval_df = merged_df[~merged_df['predicted_label'].isna() & ~merged_df['true_label'].isna()]
y_true = eval_df['true_label'].astype(int)
y_pred = eval_df['predicted_label'].astype(int)

cm = confusion_matrix(y_true, y_pred, labels=labels)
f1 = f1_score(y_true, y_pred, labels=labels, average='macro')

print("\nConfusion matrix (table):\n")
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
display(cm_df)

print("\nMacro F1 score: {:.4f}".format(f1))
print("\nDetailed classification report:\n")
print(classification_report(y_true, y_pred, labels=labels, target_names=label_names))

# Blue Confusion Matrix Plot
plt.figure(figsize=(8,6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=True, linewidths=.5, square=False)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.yticks(rotation=0)
plt.xticks(rotation=25)
plt.tight_layout()
plt.show()

# # Show omitted rows if any
# if n_missing > 0:
#     print("\nSample of omitted rows (no match_verdict):")
#     display(merged_df[merged_df['match_verdict'].isna()].head())

In [ ]:
# Cell 1: True label 2, prediction available
df_true2_pred = merged_df[(merged_df["score_x"] == 2) & (merged_df["predicted_label"] == 0)]
display(df_true2_pred)
print(f"Rows: {len(df_true2_pred)}")

In [ ]:
# Cell 1: True label 2, prediction available
df_true2_pred = merged_df[(merged_df["score_x"] == 2) & (merged_df["predicted_label"] == 2)]
display(df_true2_pred)
print(f"Rows: {len(df_true2_pred)}")

In [ ]:
# Cell 2: True label 1, prediction available
df_true1_pred = merged_df[(merged_df["score_x"] == 1) & (merged_df["predicted_label"] == 1)]
display(df_true1_pred)
print(f"Rows: {len(df_true1_pred)}")

In [ ]:
# Cell 3: True label 0, prediction available
df_true0_pred = merged_df[(merged_df["score_x"] == 0) & (merged_df["predicted_label"] == 0)]
display(df_true0_pred)
print(f"Rows: {len(df_true0_pred)}")

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# Only use rows with non-missing score_y and score_x
yx = merged_df.loc[(~merged_df['score_y'].isna()) & (~merged_df['score_x'].isna()), ['score_y', 'score_x']]
score_y = yx['score_y'].values.astype(int)  # model output
score_x = yx['score_x'].values.astype(int)  # true labels

results = []
for cut1 in range(1, 100):
    for cut2 in range(cut1 + 1, 101):
        preds = np.digitize(score_y, bins=[cut1, cut2])  # (0, 1, 2)
        macro_f1 = f1_score(score_x, preds, labels=[0, 1, 2], average="macro")
        f1_1 = f1_score(score_x, preds, labels=[1], average=None)[0]
        f1_2 = f1_score(score_x, preds, labels=[2], average=None)[0]
        results.append({
            "cut1": cut1,
            "cut2": cut2,
            "macro_f1": macro_f1,
            "f1_1": f1_1,
            "f1_2": f1_2
        })

results_df = pd.DataFrame(results)

# Find maxima
best_macro = results_df.loc[results_df['macro_f1'].idxmax()]
best_1 = results_df.loc[results_df['f1_1'].idxmax()]
best_2 = results_df.loc[results_df['f1_2'].idxmax()]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cut_labels = [
    "0) Would not refer",
    "1) Would consider referral",
    "2) Highly likely to refer"
]

def display_results_with_plot(cut1, cut2, label, f1val):
    preds = np.digitize(score_y, bins=[cut1, cut2])
    print(f"\nBest cutpoints for {label}: cut1={cut1}, cut2={cut2}, F1={f1val:.4f}")
    print(classification_report(
        score_x, preds, labels=[0, 1, 2],
        target_names=cut_labels
    ))
    cm = confusion_matrix(score_x, preds, labels=[0, 1, 2])
    cm_df = pd.DataFrame(cm, index=cut_labels, columns=cut_labels)
    print("Confusion Matrix:")
    display(cm_df)

    # Blue Confusion Matrix Plot
    plt.figure(figsize=(8,6))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=True, linewidths=.5, square=False)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title(f'Confusion Matrix ({label})')
    plt.yticks(rotation=0)
    plt.xticks(rotation=25)
    plt.tight_layout()
    plt.show()

# Show results and blue CM for each optimal cutpoint set
display_results_with_plot(int(best_macro.cut1), int(best_macro.cut2), "macro F1", best_macro.macro_f1)
display_results_with_plot(int(best_1.cut1), int(best_1.cut2), "score 1 (Would consider)", best_1.f1_1)
display_results_with_plot(int(best_2.cut1), int(best_2.cut2), "score 2 (Highly likely)", best_2.f1_2)